In [2]:
%pip install selenium

  Using cached selenium-4.31.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached trio-0.29.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached selenium-4.31.0-py3-none-any.whl (9.4 MB)
Using cached trio-0.29.0-py3-none-any.whl (492 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import asyncio
import time
from pathlib import Path
from types import SimpleNamespace
from typing import Optional, List
from dataclasses import dataclass, field

import header
from header import __root__

# External modules
from src import gs
from src.suppliers.suppliers_list import *
from src.suppliers.get_graber_by_supplier  import get_graber_by_supplier_prefix, get_graber_by_supplier_url
from src.webdriver.driver import Driver
from src.webdriver.chrome import Chrome
from src.llm.gemini import GoogleGenerativeAi
from src.llm.openai.model import OpenAIModel
from src.endpoints.prestashop.product import PrestaProduct
from src.endpoints.prestashop.language import PrestaLanguage
from src.endpoints.prestashop.product_fields import ProductFields
from src.endpoints.advertisement.facebook.scenarios.post_message import (
    post_message,
)
from src.utils.file import read_text_file, save_text_file, get_filenames_from_directory

from src.utils.jjson import j_loads, j_loads_ns, j_dumps
from src.utils.image import get_image_bytes, get_raw_image_data
from src.logger.logger import logger

ModuleNotFoundError: No module named 'selenium'

In [ ]:
async def process_suppliers(self, supplier_prefix: Optional[str | List[str, str]] = '') -> bool:
    """
    Process suppliers based on the provided prefix.
    Args:
        supplier_prefix (Optional[str | List[str, str]], optional): Prefix for suppliers. Defaults to ''.
    Returns:
        bool: True if processing is successful, False otherwise.
    Raises:
        Exception: If any error occurs during supplier processing.
    """
    try:
        if supplier_prefix:
            supplier_prefix = supplier_prefix if isinstance(supplier_prefix, list) else [supplier_prefix]
        else:
            supplier_prefix = Config.suppliers

        for prefix in supplier_prefix:
            graber: 'Graber' = get_graber_by_supplier_prefix(prefix)
            if not graber:
                logger.warning(f'No grabber found for prefix: {prefix}')
                continue
            await graber.process_scenarios_async()
            logger.info(f'Processing supplier with prefix: {prefix}')
            graber.process_supplier_scenarios_async()
    except Exception as ex:
        logger.error(f'Error while processing suppliers: {ex}', exc_info=True)
        return False
